In [88]:
import pandas as pd
import numpy as np
from math import factorial

In [89]:
train = pd.read_csv('train.csv')
train.head(2)

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev
0,1,33,80.0,0.625,0.375,0.0,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.068,3.4387
1,2,194,80.0,0.625,0.375,0.0,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.249,2.9210


In [90]:
test = pd.read_csv('test.csv')
test.head(2)

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree
0,1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006
1,2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015


In [136]:
test['potential'] = 0.0
test['pack_dens'] = 0.0
test['volume'] = 0.0
test['group15'] = 0
test['group17'] = 0
test['int_atm_dist'] = 0.0

In [137]:
def extractor(iters,strs):
    xyz = list()
    with open(strs) as f:
        for i,line in enumerate(f):
            if i > 5:
                xyz.append(line.split(' ')[1:5])
                
    data = pd.DataFrame(xyz,columns=['x','y','z','atom'])
    for i,row in data.iterrows():
        data.at[i,'atom'] = row['atom'].split('\n')[0]
    
    data.loc[data['atom'] != 'O', 'atom'] = 3
    data.loc[data['atom'] == 'O', 'atom'] = -3
    
    l = len(data)
    pot = 0
    pack_dens = 0
    vol = 0
    
    for i, row in data.iterrows():
        vol += abs((float(row['x'])**3)) + abs((float(row['y'])**3)) + abs((float(row['z'])**3))
    vol = vol/len(data)

    for i in range(l):
        for j in range(i+1,l):
            xdiff =  (float(data.iloc[i]['x']) - float(data.iloc[j]['x']))
            ydiff =  (float(data.iloc[i]['y']) - float(data.iloc[j]['y']))
            zdiff =  (float(data.iloc[i]['z']) - float(data.iloc[j]['z']))
            pot += (data.iloc[i]['atom'] * data.iloc[j]['atom'])/(np.sqrt(xdiff**2 + ydiff**2 + zdiff**2))
            pack_dens += np.sqrt(xdiff**2 + ydiff**2 + zdiff**2)

    pack_dens = pack_dens / (factorial(len(data))/(factorial(2)*factorial(len(data)-2)))


    pos_pair = data[data['atom']==3]
    neg_pair = data[data['atom']==-3]
    int_atom_dist = 0
    poss = len(pos_pair)
    negg = len(neg_pair)
    
    for i,rowp in pos_pair.iterrows():
        for i,rown in neg_pair.iterrows():
            xx = (float(rowp['x']) - float(rown['x']))**2
            yy = (float(rowp['y']) - float(rown['y']))**2
            zz = (float(rowp['z']) - float(rown['z']))**2
            int_atom_dist += np.sqrt(xx + yy + zz)
    
    int_atom_dist = int_atom_dist/(len(pos_pair) * len(neg_pair))
    
    test.at[iters-1,'potential'] = pot
    test.at[iters-1,'pack_dens'] = pack_dens
    test.at[iters-1,'volume'] = vol
    test.at[iters-1,'group15'] = poss
    test.at[iters-1,'group17'] = negg
    test.at[iters-1,'int_atm_dist'] = int_atom_dist

In [138]:
for  i in range(len(test)):
    iters = i+1
    strs = 'test/test/'+str(iters)+'/geometry.xyz'
    extractor(iters,strs)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [142]:
test.to_csv('test_v2.csv',index=False)